# For DL based on Gramian Angular Field in All Data

# Pickle Path Extraction

In [77]:
import pandas as pd
import os
import numpy as np
import pickle
import torch
from typing import TypeVar, Tuple, List, Union, Dict, Callable, Optional


# Check Diagnosis With SpreadSheet

In [34]:
__Version__ = '0.0.2'
__author__ = 'Won sik jung'
'''S_Data Based on R/T Tangent images map'''

origin_path = '/home/yongho.choi/Data/severance/processed/'


Healthy = [str(origin_path)+x+'/PARAMETER4/' for x in os.listdir(origin_path) if 'Healthy' in x]
patient_path = origin_path+"patient/PARAMETER4/"


In [55]:
# Extraction Pickle path in Some Path
# label= 0 or 1
# 0 = 건강인(Healthy), 1 = 환자(Patient)

def extrack_pickle(path,names=None,option=None):
    result=list()
    for path, dirs, files in os.walk(path):
        for file in files:
            name, ext = os.path.splitext(file)
            
            # if ext == '.pickle' and 'Images' in name:
            if ext == '.pickle' and 'Images' in name:

                # find folder
                if names==None:
                    # file_path = os.path.join(path,file)
                    file_path = os.path.join(path,file)+'label='+str(option)

                    result.append(file_path)

                # find folder Contain names
                elif str(names) in name:
                    # file_path = os.path.join(path,file)
                    file_path = os.path.join(path,file)+'label='+str(option)

                    result.append(file_path)

                else:
                    pass

    return sorted(result)

#모든 건강인의 1번 bin 파일에 0label 추가

Path_healthy_image=list()

for path in Healthy:
    Path_healthy_image.extend(extrack_pickle(path,'_1_','0'))

#모든 환자군의 1번 bin 파일에 1label 추가
Path_patient_image = extrack_pickle(patient_path,'_1_','1')

#모든 1번 파일을 path_total_fieldmap에 적재
path_total_image = Path_healthy_image + Path_patient_image


In [36]:
len(path_total_fieldmap)

1383

In [37]:
print('환자경로{0}, 건강인 폴더{1}\n'.format(patient_path,Healthy))
print('건강인(Healthy) = {0},환자(Patient) = {1}, 전체(Total) = {2}'\
       .format(len(Path_healthy_image),len(Path_patient_image),len(path_total_fieldmap)))


환자경로/home/yongho.choi/Data/severance/processed/patient/PARAMETER4/, 건강인 폴더['/home/yongho.choi/Data/severance/processed/Healthy_hospital/PARAMETER4/', '/home/yongho.choi/Data/severance/processed/Healthy_adenosin/PARAMETER4/']

건강인(Healthy) = 137,환자(Patient) = 1246, 전체(Total) = 1383


# Pickle Load From Extracted Pickle file 

In [38]:
# Make Load,Save pickle function

def save_pickle(fname, dict):
    with open(fname,'wb') as fw:
        pickle.dump(dict,fw)
    fw.close()

def load_pickle(fname):
    with open(fname,'rb') as fr:
        dict = pickle.load(fr)
    return dict 


# 0번째 피클 파악

pickle1= load_pickle(str(Path_patient_image[0]).split('label=')[0])
pickle1.keys()
# Keys Has 2 features s.t dict_keys(['Recurrence_plot', 'GASF'])

pickle1['GASF'].shape
# # 46*46


(46, 46)

In [39]:
# import matplotlib.pyplot as plt

# x = pickle1['GASF'][0]
# y = pickle1['GASF'][1]
# # plt.plot(pickle1['GASF'])

# plt.mesh(x)
# plt.show()

In [50]:
path = str(Path_patient_image[0])
           
pickle_data= load_pickle(str(path.split('label=')[0]))

# Extract Value
Recur_plot,GASF = pickle_data['Recurrence_plot'],pickle_data['GASF']

#R,T 이미지 적재 및 label 추출
x = np.stack([Recur_plot,GASF])
label = int(path.split('label=')[1])


In [62]:
print(Recur_plot.shape)
print(GASF.shape)

print(x.shape)


(46, 46)
(46, 46)
(2, 46, 46)


In [63]:
# Make 2 Data, GASF _ RT & LabelRecur_plot
Total_Recur = list()
Total_GASF = list()

Total_x=list()
Total_label=list()


for path in path_total_image:
    # split[0] = path, split[1]= label
    # 0번 경로는 label= 전은 패킷 경로, lael= 후는 label 정보
    try:
        pickle_data= load_pickle(str(path.split('label=')[0]))

        # Extract Value
        Recur_plot,GASF = pickle_data['Recurrence_plot'],pickle_data['GASF']

        #R,T 이미지 적재 및 label 추출
        x = np.stack([Recur_plot,GASF])
        label = int(path.split('label=')[1])

        #학습용 데이터로 활용
        # append Each Value
        Total_x.append(x)
        Total_label.append(label)

    except:
        print(str(path[0].split('label=')[0]))

T_label = [[x] for x in Total_label]


In [67]:
print("X shape = {0}, Y shape ={1}".format(np.array(Total_x).shape,np.array(T_label).shape))
print('Y Example = {0}'.format(T_label[:10]))

data_origin = {'features':Total_x,'labels':T_label}
data_origin.keys()

X shape = (1383, 2, 46, 46), Y shape =(1383, 1)
Y Example = [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]


dict_keys(['features', 'labels'])

# Make Dataset, DataLoader & Shuffle, Split

In [79]:
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms

class CustomDataset(Dataset):
    def __init__(self,Dataset):
        # self.Dataset = Dataset
        self.features =  Dataset['features']
        self.labels = Dataset['labels']

        self.x = self.features
        self.y = self.labels

    # 생성자, 데이터를 전처리 하는 부분   

    def __len__(self):
        return len(self.y)
    # 데이터셋의 총 길이를 반환하는 부분   

    def __getitem__(self,idx):

        x = torch.FloatTensor(self.x[idx])
        y = torch.FloatTensor(self.y[idx])
        return x,y
    # idx(인덱스)에 해당하는 입출력 데이터를 반환
    
    def __label__(self):
        return self.labels


In [80]:
dataset = CustomDataset(data_origin)
seed = 1004
batch_size = 8


#               -Data split(train:0.7%(672), test:0.3*0.5=0.15%(144), validation:0.3*0.5=0.15%(145))-
#               -Stratify에 설정으로, label 비율 유지( 정상인: 건강인 14%:86%)
train_set, test_data0 = train_test_split(dataset, train_size = 0.7, test_size = 0.3, random_state = seed, stratify = dataset.labels)

#               -strafify 옵션 설정을 위해 test_set0 생성후 label에 적용 추출

test_set0 = CustomDataset({'features' : [x[:][0] for x in test_data0], 'labels': [x[:][1] for x in test_data0] })
validation_set, test_set = train_test_split(test_set0, train_size = 0.5, test_size = 0.5, random_state = seed, stratify = test_set0.labels)

#               -splited data를 Model에 사용하기 편하게 조절(splited데이터를 Dataloader로)-
#               -From splited data To Dataloader-

trainloader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, shuffle = True)
testloader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = False)
validationloader = torch.utils.data.DataLoader(validation_set, batch_size = batch_size, shuffle = False)



/home/wonsik.jung/anaconda3/envs/tenpy/lib/python3.9/site-packages/sklearn/utils/validation.py:856: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  array = np.asarray(array, order=order, dtype=dtype)
/home/wonsik.jung/anaconda3/envs/tenpy/lib/python3.9/site-packages/sklearn/utils/validation.py:856: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = np.asarray(array

In [81]:
print(len(train_set))
print(len(validation_set))
print(len(test_set))

968
207
208


# Model Select & Tuning

In [82]:
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optimize
import torch.nn.functional as F
from torch.autograd import Variable
import sklearn

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.IMAGENET1K_V1')

model.features[0][0] = nn.Conv2d(in_channels = 2,out_channels = 32,
                    kernel_size=3, stride =2, padding = 1)

model.classifier = nn.Sequential(
                nn.Dropout(p=0.2, inplace=True),
                nn.Linear(in_features = 1280, out_features= 1, bias=True),
                nn.Sigmoid()
                )




In [83]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_b0(weights='EfficientNet_B0_Weights.IMAGENET1K_V1')

model.features[0][0] = nn.Conv2d(in_channels = 2,out_channels = 32,
                    kernel_size=3, stride =2, padding = 1)

model.classifier = nn.Sequential(
                nn.Dropout(p=0.2, inplace=True),
                nn.Linear(in_features = 1280, out_features= 1, bias=True),
                nn.Sigmoid()
                )



In [84]:
model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivation(
       

# Define (Traing, Test) And Run Epoch

In [85]:
from tqdm import tqdm
import time
import sklearn
from torch.utils.tensorboard import SummaryWriter

model=model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-3)
criterion = nn.BCELoss()

save_path="//home//wonsik.jung//Vscode test//GAF&Recur_DL_Model"
model_save_path = str(save_path)+"/savedmodel.pth"

train_writer = SummaryWriter(str(save_path)+'//Loss//Train', comment = 'Train')
validation_writer = SummaryWriter(str(save_path)+'//Loss//Validation', comment = 'Validation')



In [86]:
#                            --------(3-1)Define Train 정의-------
def train(trainloader, model, optimizer, device,trainwriter=None, epoch: Optional[int] =None):
    model.train()
    with tqdm(total = len(trainloader), desc = 'Training Processing') as pbar:
        for batch_idx,(data, label) in enumerate(trainloader):
            data, label = data.to(device), label.to(device)
            # Set optimizer_Gradient()
            # Set Forward + Backward + Optimize

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output,label)
            loss.backward()
            optimizer.step()
            pbar.update()

        if trainwriter != None:
            train_writer.add_scalar('Train_Loss', loss,epoch+1)



#                            --------(3-2)Define validation,test 정의-------
def val_test(dataloader, model, device, val_testwriter=None, epoch: Optional[int] =None):
    model.eval()
    loss = 0
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(dataloader):
            data, label = data.to(device), label.to(device)
            output = model(data)
            pred = torch.round(output)
            loss += criterion(output, label).item()
            #   -------- BinaryClass므로 round를 넣어 0또는 1로 결과 예측----
            #   -------- pred_result와 label을 batch 마다 적재하는 과정-----
            label_cpu, pred_cpu, output_cpu = label.detach().cpu().numpy(), pred.detach().cpu().numpy(), output.detach().cpu().numpy()

            if batch_idx == 0:
                label_result, pred_result, proba_result = label_cpu, pred_cpu, output_cpu
            else:
                label_result, pred_result, proba_result = np.concatenate([label_result, label_cpu]), np.concatenate([pred_result, pred_cpu]), np.concatenate([proba_result, output_cpu])

        if val_testwriter != None:
            val_testwriter.add_scalar('validation_Loss', loss,epoch+1)
            val_testwriter.add_graph(model,data)


    return loss, label_result, pred_result, proba_result


In [87]:

def run_epoch(model, optimizer, device, trainloader, validationloader, 
              epochs: int, patience: Optional[int]=4, model_save_path: Optional[str]=None, 
              train_writer=None, val_testwriter=None, scheduler=None):
    cnt = 0
    for epoch in range(epochs):
        ###       ------Train Proecess
        train(trainloader, model, optimizer, device, trainwriter=train_writer, epoch=epoch)

        ###       -----Validation Process
        val_loss, label_result, pred_result, proba_result = val_test(validationloader, model, device, val_testwriter=validation_writer, epoch=epoch)
 
        # validation_writer.add_scalar('Validaion_loss', validation_loss,epoch+1)
            

        ###   -------- Early Stop & Model Save
        try:
            if min_loss == None:
                min_loss = val_loss
        except:
            min_loss = val_loss

        if val_loss > min_loss:
            cnt += 1
            if cnt >= patience:
                break

        else:
            try:
                min_loss = np.min(min_loss,val_loss)
                cnt = 0
            except:
                pass
            
    ###  ------ scheduler Apply
    if scheduler!= None:
        scheduler.step()


    ###     ----if epoch Done!, save model state_dict
    if model_save_path!= None:
        torch.save(model.state_dict(),model_save_path)


    acc = sklearn.metrics.accuracy_score(label_result,pred_result)
    print('\nFinal Epoch:{} 평균 loss: {:.9f}, 정확도: {:.3f})\n'.format(epoch+1,val_loss,acc))



In [88]:
epochs= 100

# validationloader, train_writer, val_testwriter, epoch


# run_epoch(model, optimizer, device, trainloader, validationloader, epochs,patience=5, model_save_path=model_save_path, train_writer=train_writer, val_testwriter=validation_writer)



Training Processing: 100%|██████████| 121/121 [00:10<00:00, 11.03it/s]



Final Epoch:19 평균 loss: 9.733301514, 정확도: 0.903)



# Check Saved Model Perfomance

In [ ]:
model.load_state_dict(torch.load(save_path+'/savedmodel.pth'))
                                 
label_result, pred_result = test(testloader,model,device)

acc = sklearn.metrics.accuracy_score(label_result,pred_result)
roc = sklearn.metrics.roc_auc_score(label_result,pred_result)
pre = sklearn.metrics.precision_score(label_result,pred_result)
sen = sklearn.metrics.recall_score(label_result,pred_result)
f1 = sklearn.metrics.f1_score(label_result,pred_result)

print('정확도:{:.3}, ROC:{:.3f},정밀도:{:.3f}, 민감도:{:.3f}, F1 점수:{:.3f}'.format(acc,roc,pre,sen,f1))

정확도:0.883, ROC:0.680,정밀도:0.909, 민감도:0.960, F1 점수:0.934


In [ ]:
print(pred_result.shape)
sum(pred_result)

print()

(145, 1)


array([132.], dtype=float32)

In [90]:
def performance_score(label,predict):
    '''
    Input
    label = model.label
    proba = model.predict_proba(test)
    
    Output

    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)  # sensitivity
    f1 = f1_score(y_test, pred)
    '''

    #####from confusion matrix calculate accuracy
    CM = sklearn.metrics.confusion_matrix(label,predict)
    TN, FP, FN, TP = CM[0,0], CM[0,1], CM[1,0], CM[1,1]

    specificity = TN/(FP+TN)

    precision = TP/(TP+FP)
    recall = TP/(TP+FN) # sensitivity

    accuracy=(TP+TN)/(TN+FP+FN+TP)
    F1 = 2*(precision*recall)/(precision+recall)

    return F1, accuracy, precision, recall, specificity



def opt_threshold(label: np.array, proba: np.array) -> list:
    '''
    Input

    label = model.label
    proba = model.predict_proba(test)

    Output

    roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i),
    '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i),
    'thresholds' : pd.Series(thresholds, index = i)})
    '''

    fpr, tpr, thresholds = sklearn.metrics.roc_curve(label, proba)

    i = np.arange(len(tpr)) # index for df
    roc_info = pd.DataFrame([fpr, tpr, 1-fpr, tpr-(1-fpr), thresholds], index=['fpr','tpr','1-fpr','tf','thresholds']).T
    cutoff_info = roc_info.iloc[(roc_info.tf).abs().argsort()[:1]]
    
    return np.round(cutoff_info['thresholds'].tolist()[0],4)

def evaluate_prediction(label, proba, cutoff=1):
    '''  
    Input

    label = model.label
    proba = model.predict_proba(test)
    
    Output

     accuracy = accuracy_score(y_test, pred)
     precision = precision_score(y_test, pred)
     recall = recall_score(y_test, pred)  # sensitivity
     f1 = f1_score(y_test, pred)
     '''

    
    if cutoff==1:
        thr = opt_threshold(label,proba)
        pred = proba.copy()
        pred[pred<thr] = 0
        pred[pred>=thr] = 1
    else:
        thr = cutoff
        pred = proba.copy()
        pred[pred<thr] = 0
        pred[pred>=thr] = 1


    #####from confusion matrix calculate accuracy
    CM = sklearn.metrics.confusion_matrix(label,pred)
    TN, FP, FN, TP = CM[0,0], CM[0,1], CM[1,0], CM[1,1]

    specificity = TN/(FP+TN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN) # sensitivity
    accuracy=(TP+TN)/(TN+FP+FN+TP)
    F1 = 2*(precision*recall)/(precision+recall)

    # fpr, tpr, _ = roc_curve(label, model.predict_proba(test)[:,1])
    auc = sklearn.metrics.roc_auc_score(label, proba)

    return F1, accuracy, auc, precision, recall, specificity

In [101]:
model.load_state_dict(torch.load(model_save_path))

_, test_label, test_predict, test_proba = val_test(testloader, model, device)


# F1, accuracy, precision, recall, specificity = performance_score(test_label,test_predict)
Cut = opt_threshold(test_label, test_proba)
Result = evaluate_prediction(test_label, test_proba, cutoff = Cut)


# return F1, accuracy, auc, precision, recall, specificity
print('F1:{0:.3f}, Acc:{1:.3f}, Auc:{2:.3f}, Precision:{3:.3f}, Recall:{4:.3f}, Specificity:{5:.3f}'
      .format(Result[0], Result[1], Result[2], Result[3], Result[4], Result[5]))


#efficiet0.V1
'F1:0.953, Acc:0.918, Auc:0.923, Precision:0.989, Recall:0.920, Specificity:0.905'



F1:0.953, Acc:0.918, Auc:0.923, Precision:0.989, Recall:0.920, Specificity:0.905


'F1:0.953, Acc:0.918, Auc:0.923, Precision:0.989, Recall:0.920, Specificity:0.905'